In [1]:
import os
import urllib.request
import zipfile

import micronas
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import class_weight
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical

2025-12-25 17:44:54.152735: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766709894.174679  149059 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766709894.181146  149059 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1766709894.199389  149059 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766709894.199442  149059 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1766709894.199445  149059 computation_placer.cc:177] computation placer alr

In [2]:
import warnings

warnings.filterwarnings("ignore")

# Loading AReM dataset

In [3]:
url = "https://cdn.uci-ics-mlr-prod.aws.uci.edu/366/activity%2Brecognition%2Bsystem%2Bbased%2Bon%2Bmultisensor%2Bdata%2Bfusion%2Barem.zip"
zip_path = "AReM.zip"
root_dir = "AReM"

if not os.path.exists(zip_path):
    urllib.request.urlretrieve(url, zip_path)

if not os.path.exists(root_dir):
    with zipfile.ZipFile(zip_path, "r") as z:
        z.extractall(root_dir)

data_dir = os.path.join(root_dir, "")

activities = sorted(
    d for d in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, d))
)
label_map = {name: i for i, name in enumerate(activities)}

X = []
y = []

for act in activities:
    act_dir = os.path.join(data_dir, act)
    for f in sorted(os.listdir(act_dir)):
        if f.endswith(".csv"):
            path = os.path.join(act_dir, f)
            df = pd.read_csv(path, skiprows=4, on_bad_lines="skip")
            # drop time column
            signal = df.iloc[:, 1:].to_numpy(dtype=np.float32)

            X.append(signal)
            y.append(np.full((signal.shape[0], 1), label_map[act]))

X = np.vstack(X)
y = np.vstack(y)
X = np.nan_to_num(X, nan=0.0)
X.shape, y.shape

((42237, 6), (42237, 1))

# Sliding Window

In [4]:
def sliding_window(X, y, window_size=50, step=25, num_classes=None):
    """
    X: (N,6)
    y: (N,)
    Returns:
        X_win: (num_windows, window_size, 6)
        y_win: (num_windows,)
    Label for window = mode of y in window
    """
    from scipy.stats import mode

    N = X.shape[0]
    X_win, y_win = [], []

    for start in range(0, N - window_size + 1, step):
        end = start + window_size
        X_win.append(X[start:end])
        y_win.append(mode(y[start:end])[0][0])

    if num_classes is None:
        num_classes = np.max(y) + 1

    y_onehot = to_categorical(y_win, num_classes=num_classes)
    return np.stack(X_win), y_onehot

In [5]:
X_windows, y_windows = sliding_window(X, y, window_size=50, step=25)

X_train, X_test, y_train, y_test = train_test_split(
    X_windows, y_windows, test_size=0.2, random_state=0, stratify=y_windows
)

X_train, X_valid, y_train, y_valid = train_test_split(
    X_train, y_train, test_size=0.1875, random_state=0, stratify=y_train
)

X_train.shape, y_train.shape, X_valid.shape, y_valid.shape, X_test.shape, y_test.shape

((1096, 50, 6), (1096, 7), (254, 50, 6), (254, 7), (338, 50, 6), (338, 7))

# Computing Class Weight

In [6]:
labels = y_train.argmax(1).flatten()
labels_unique = np.unique(labels)
class_weights = class_weight.compute_class_weight(
    "balanced", classes=labels_unique, y=labels
)
class_weights = dict(zip(labels_unique, class_weights))
class_weights

{0: 1.799671592775041,
 1: 2.115830115830116,
 2: 0.8372803666921314,
 3: 0.8372803666921314,
 4: 0.8372803666921314,
 5: 0.8372803666921314,
 6: 0.8372803666921314}

# Defining the NAS

In [7]:
@tf.keras.utils.register_keras_serializable(package="ConditionalPoolingLayer1D")
class ConditionalPoolingLayer1D(layers.Layer):
    def __init__(self, pool_size=2, pool_type="max", **kwargs):
        super().__init__(**kwargs)
        self.pool_size = pool_size
        self.pool_type = pool_type

        if pool_type == "max":
            self.pooling_layer = layers.MaxPooling1D(pool_size=pool_size)
        else:
            self.pooling_layer = layers.AveragePooling1D(pool_size=pool_size)

    def call(self, inputs):
        input_shape = tf.shape(inputs)[1]
        output_length = input_shape // self.pool_size

        return tf.cond(
            output_length > 0, lambda: self.pooling_layer(inputs), lambda: inputs
        )

    def get_config(self):
        config = super().get_config()
        config.update(
            {
                "pool_size": self.pool_size,
                "pool_type": self.pool_type,
            }
        )
        return config

In [8]:
class IMUModel(micronas.HyperModel):
    def build(self, hp):
        time_domain = keras.Sequential()

        time_domain.add(keras.Input(shape=X_train.shape[1:], name="input"))

        time_domain.add(layers.BatchNormalization())

        for Block in range(3):
            if hp.Boolean("Block%d" % Block, True):
                time_domain.add(
                    layers.Conv1D(
                        hp.Int("conv%d" % Block, min_value=1, max_value=500, step=10),
                        kernel_size=(
                            hp.Int(
                                "kernel_size%d" % Block,
                                min_value=1,
                                max_value=8,
                                step=1,
                            ),
                        ),
                        activation=hp.Choice(
                            "activation%d" % Block, ["relu", "tanh", "sigmoid"]
                        ),
                        padding=hp.Choice("padding%d" % Block, ["same", "valid"]),
                        strides=hp.Int(
                            "stride%d" % Block, min_value=1, max_value=2, step=1
                        ),
                    )
                )
                if hp.Boolean("Block%d_BN" % Block, True):
                    time_domain.add(layers.BatchNormalization())
                if hp.Boolean("Block%d_pooling" % Block, True):
                    if hp.Boolean("Block%d_MaxPooling" % Block, True):
                        time_domain.add(
                            ConditionalPoolingLayer1D(
                                hp.Choice("pool%d" % Block, [2]), pool_type="max"
                            )
                        )
                    else:
                        time_domain.add(
                            ConditionalPoolingLayer1D(
                                hp.Choice("pool%d" % Block, [2]), pool_type="average"
                            )
                        )

        # prefinal Block
        if hp.Boolean("prefinal_Global", True):
            time_domain.add(layers.GlobalAveragePooling1D())
        if hp.Boolean("prefinal_Dropout", True):
            time_domain.add(
                layers.Dropout(hp.Float("dropout", min_value=0.4, max_value=0.5))
            )
        time_domain.add(layers.Flatten(name="flatten"))

        # Final Block
        for Block in range(3):
            if hp.Boolean("finalBlock%d" % Block, True):
                if hp.Boolean("finalBlock%d_BN" % Block, True):
                    time_domain.add(layers.BatchNormalization())
                if hp.Boolean("finalBlock%d_dense" % Block, True):
                    time_domain.add(
                        layers.Dense(
                            hp.Int(
                                "dense_size%d" % Block,
                                min_value=3,
                                max_value=1024,
                                step=20,
                            ),
                            activation=hp.Choice(
                                "finalactivation%d" % Block, ["relu", "tanh", "sigmoid"]
                            ),
                        )
                    )
                if hp.Boolean("final_Dropout%d" % Block, True):
                    time_domain.add(
                        layers.Dropout(
                            hp.Float(
                                "final_dropout%d" % Block, min_value=0.4, max_value=0.5
                            )
                        )
                    )
        time_domain.add(
            layers.Dense(
                y_train.shape[1],
                activation="softmax",
                kernel_regularizer=keras.regularizers.l1(
                    hp.Float("l1", min_value=1e-5, max_value=1e-3, step=1e-6)
                ),
            )
        )

        learning_rate = hp.Float("lr", min_value=1e-6, max_value=1e-2, sampling="log")
        time_domain.compile(
            optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
            loss="categorical_crossentropy",
            metrics=[
                "accuracy",
                tf.keras.metrics.Precision(name="precision"),
                tf.keras.metrics.Recall(name="recall"),
                tf.keras.metrics.AUC(name="auc", curve="PR"),
            ],
        )
        return time_domain

    def fit(self, hp, model, *args, **kwargs):
        hist = model.fit(*args, **kwargs)

        model_score = np.mean(hist.history["val_auc"])
        objective = model_score
        hist.history["objective"] = objective
        return hist.history

In [9]:
tuner_time = micronas.RandomSearch(
    IMUModel(),
    objective=micronas.Objective("objective", direction="max"),
    max_trials=1,
    overwrite=True,
    max_model_size=80000,
    max_consecutive_failed_trials=float("inf"),
)
tuner_time.search_space_summary()

W0000 00:00:1766709899.830137  149059 gpu_device.cc:2341] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


Search space summary
Default search space size: 56
Block0 (Boolean)
{'default': True, 'conditions': []}
conv0 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 500, 'step': 10, 'sampling': 'linear'}
kernel_size0 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 8, 'step': 1, 'sampling': 'linear'}
activation0 (Choice)
{'default': 'relu', 'conditions': [], 'values': ['relu', 'tanh', 'sigmoid'], 'ordered': False}
padding0 (Choice)
{'default': 'same', 'conditions': [], 'values': ['same', 'valid'], 'ordered': False}
stride0 (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 2, 'step': 1, 'sampling': 'linear'}
Block0_BN (Boolean)
{'default': True, 'conditions': []}
Block0_pooling (Boolean)
{'default': True, 'conditions': []}
Block0_MaxPooling (Boolean)
{'default': True, 'conditions': []}
pool0 (Choice)
{'default': 2, 'conditions': [], 'values': [2], 'ordered': True}
Block1 (Boolean)
{'default': True, 'conditions': []}
conv1 (Int)


# Start NAS

In [10]:
callback = tf.keras.callbacks.EarlyStopping(monitor="val_loss", patience=10)

tuner_time.search_atleast(
    20,
    X_train,
    y_train,
    epochs=30,
    batch_size=128,
    callbacks=[callback],
    class_weight=class_weights,
    validation_data=(X_valid, y_valid),
)

Trial 103 Complete [00h 00m 01s]
objective: -inf

Best objective So Far: 0.6544561207294464
Total elapsed time: 00h 05m 20s


# Get Best Model

In [11]:
models = tuner_time.get_best_models(num_models=1)
best_estimator = models[0]

best_estimator.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ batch_normalization             │ (None, 50, 6)          │            24 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv1d (Conv1D)                 │ (None, 49, 121)        │         1,573 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 49, 121)        │           484 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conditional_pooling_layer1d     │ (None, None, 121)      │             0 │
│ (ConditionalPoolingLayer1D)     │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 121)            │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 121)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 121)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 121)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 7)              │           854 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,935 (11.46 KB)

 Trainable params: 2,681 (10.47 KB)

 Non-trainable params: 254 (1016.00 B)

# Classification Report

In [12]:
y_pred = best_estimator.predict(X_test).argmax(1)

target_names = [name for name, idx in sorted(label_map.items(), key=lambda x: x[1])]

print(classification_report(y_test.argmax(1), y_pred, target_names=target_names))
print(confusion_matrix(y_test.argmax(1), y_pred))

11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
              precision    recall  f1-score   support

    bending1       0.68      0.93      0.78        27
    bending2       0.73      0.83      0.78        23
     cycling       0.87      0.95      0.91        58
       lying       0.85      0.88      0.86        57
     sitting       0.50      0.03      0.06        58
    standing       0.56      0.93      0.70        58
     walking       0.94      0.86      0.90        57

    accuracy                           0.75       338
   macro avg       0.73      0.77      0.71       338
weighted avg       0.74      0.75      0.70       338

[[25  0  0  0  0  2  0]
 [ 2 19  0  2  0  0  0]
 [ 0  0 55  0  0  0  3]
 [ 0  2  0 50  0  5  0]
 [10  5  0  5  2 36  0]
 [ 0  0  0  2  2 54  0]
 [ 0  0  8  0  0  0 49]]


In [13]:
best_estimator.save("AReM_Best.keras")